In [ ]:
# a tracker/deterrent based on using a points system 

# structure to hold new items (name, website link, price, category, value, date logged, redeemed?, etc.)
# function to add new items 
# function to redeem item (will have to update the structure and total points)
# function to total up point value in structure
# function to search structure by name, category, value
# functions to calculate points based on current date

# function to show you what's possible to purchase based on point values today


# create item as its own class 
# init to create the base of the object (name, link, price, category, value, date logged, date redeemed, delete_ind, etc.)
# function to update data

# tracker class will create item objects, store them, calculate points, etc. 

# menu class 
# calculate points and set it upon starting
# load most recent saved data
# save the data after every session 


In [ ]:
# structure to hold items (can use pandas dfand maybe pickle the data?)
# could use pickle or json to serialize the data
# error checking for values entered for the wishlist item. maybe that should be handled with the menu?

# define the wishlist categories?
# in the future, create a way to assign the point value?

In [1]:
# import packages 
import pandas as pd
import datetime as dt
import pickle
import os
#from pathlib import Path
from sys import exit

In [4]:
class Tracker():
    
    """
    Tracker object to hold the wishlist items and track point values
    """
    
    def __init__(self, username):
        
        # creates the wishlist dict
        self.__wishlist = dict()
        # captures when the tracker was started
        self.__start_date = dt.date.today()
        # captures current date for state purposes
        self.__cur_date = dt.date.today()
        # inital points
        self.__points = 0
        # initial cost
        self.__cost = 0
        # get a username
        self.__username = username
        
    def tracker_info(self):
        
        print("Wishlist contains:", self.__wishlist)
        print("Start Date of Tracker:", self.__start_date)
        print("Username is:", self.__username)
        
    def print_wishlist(self, csv = "no"):
        
        """Function to put the wishlist dict into a df 
        for easy viewing with an option to create csv"""
        
        # have not built in csv functionality
        df = pd.DataFrame(self.__wishlist)
        df_T = df.T
        
        return df_T[(df_T["redeemed"] == "no") & (df_T["del_ind"] == "no")]
        #return df_T[df_T["redeemed"] == "no"]
    
    # getter
    @property 
    def points(self):
        return self.__points
    
    @property
    def wishlist(self):
        return self.__wishlist
        
    #@points.setter
    #def set_points(self, points):
        
        #"""After calculating the points for the day, set that as the points """
        
        #self.points = points
                
        
    def add_item(self, item, price, category, value, 
                          override_dates = "no", date = dt.date(1900, 1, 1)):
        
        """
        Function to add wishlist item if it doesn't already exist in the dict
        """

        if item not in self.__wishlist.keys():
            
            if override_dates == "no":
                self.__wishlist[item] = {"price":price, "category":category, 
                                         "value":value, "date":dt.date.today(),
                                         "redeemed_dt":dt.date(1900, 1, 1), "redeemed":"no",
                                        "del_ind": "no"}
                print("Item added to wishlist")
            elif override_dates == "yes":
                # allows overriding logging today's date in case we're backtracking
                self.__wishlist[item] = {"price":price, "category":category, 
                                         "value":value, "date":date, 
                                         "redeemed_dt":dt.date(1900, 1, 1), "redeemed":"no",
                                        "del_ind": "no"}
                print("Item added to wishlist")
                
        else:
            # revisit to create a way to bypass this upon user confirmation
            print("Item is already in wishlist")
    
    def del_item(self, item):
        
        """
        Function to mark an item as deleted
        """
        
        try:
            self.__wishlist[item]["del_ind"] = "yes"
            print("Item deleted from wishlist")
        except:
            # assuming that it's a key error
            print("Item not in wishlist")
            
                
    def update_item(self, item, to_update, update):
        
        """
        Function to update an item on the wishlist based on the key defined
        """
        
        try:
            self.__wishlist[item][to_update] = update
            print("Item has been updated")
        except:
            print("Update failed. Please check the item name or data field again.")
    
    def calc(self):
        
        """
        Function to calculate the total points and cost based on the number of non-redeemed items current in the list
        """
        
        points = 0
        cost = 0
        
        for item in self.__wishlist.keys():
            temp = self.__wishlist[item]
            
            # for items that are active and haven't been redeemed or deleted
            if temp["date"] <= dt.date.today() and temp["redeemed"] == "no" and temp["del_ind"] == "no":
                
                points += abs(dt.date.today() - temp["date"]).days
                cost += temp["price"]
            
            # for items that have been redeemed, add back the value up until redeemed
            # then subtracted the value redeemed
            if temp["redeemed_dt"] > dt.date(1900, 1, 1) and temp["redeemed_dt"] <= dt.date.today() and temp["del_ind"] == "no":

                points += abs(temp["redeemed_dt"] - temp["date"]).days
                points -= temp["value"]
        
        # set points attribute
        self.__points = points
        self.__cost = cost
                
        return points, cost
    
    
    def redeem_item(self, item):
        
        """
        Function to mark an item as redeemed in the wishlist
        """
        try:
            self.__wishlist[item]["redeemed"] = "yes"
            self.__wishlist[item]["redeemed_dt"] = dt.date.today()
            self.__points -= self.__wishlist[item]["value"]
            print("Item marked as redeemed")
        except:
            # assuming that it's a key error
            print("Item not in wishlist")

    
    
    def search_wishlist(self):
        
        # may have to be broken down into other methods depending on search?
        
        pass
        

In [3]:
class Menu():
    
    """
    User interface for the Impulse Tracker
    """
    # hold all the tracker names that are created
    tracker_names = set()
    
    def __init__(self):
        self.cur_tracker = None
        self.cur_username = ''
    
    def display_main(self):
        """ 
        Displays the main menu of creating a new tracker, loading previous tracker, or quit 
        """
                
        while True:
            print("""
                     __             __   ___     __   __   ___       __          __  
            |  |\/| |__) |  | |    /__` |__     /__` |__) |__  |\ | |  \ | |\ | / _` 
            |  |  | |    \__/ |___ .__/ |___    .__/ |    |___ | \| |__/ | | \| \__> 

            ___  __        __        ___  __                                         
             |  |__)  /\  /  ` |__/ |__  |__)                                        
             |  |  \ /~~\ \__, |  \ |___ |  \     
            """)

            print("""
            Welcome to Impulse Spending Tracker, where you can create a wishlist and curb
            impulse spending using a point system. The longer items stay un-redeemed on 
            your wishlist, the more points you accumulate. Accumulate enough points and you
            can redeem items from your wishlist. 

            What would you like to do?

            1) Create a new tracker
            2) Load a saved tracker
            3) Quit
            """)

            resp = input("Enter your response here: ")

            if resp == '1':
                self.create_tracker()
            elif resp == '2':
                self.load_tracker()
            elif resp == '3':
                self.quit()
            else:
                print("Invalid response. Please try again.")
    
    def display_post_create(self, username):
        """
        Displays the menu after you initialize a new tracker with create_tracker
        """
        
        print("""
        To get started...
        
        1) Add an item to the wishlist
        2) Quit
        
        """)
        
        while True:
            
            resp = input("What would you like to do? ")
            
            if resp == '1':
                self.add_item()
            elif resp == '2':
                # have it save by default
                self.save_tracker()

            
    
    def display_functions(self):
        """
        Displays the functional menu after you either create a new tracker and 
        add at least one item or if you load a saved tracker
        """
          
        while True:
            print("""
            Select one of the following options:

            1) Add an item to the wishlist
            2) Update an item in the wishlist
            3) Delete an item from the wishlist
            4) Redeem an item from the wishlist
            5) View wishlist
            6) View tracker info
            7) Save tracker
            8) Quit
            """)
            resp = input("What would you like to do? ")
            
            if resp == "1":
                self.add_item()
            elif resp == "2":
                self.update_item()
            elif resp == "3":
                self.del_item()
            elif resp == "4":
                self.redeem_item()
            elif resp == "5":
                self.print_wishlist()
            elif resp == "6":
                self.tracker_info()
            elif resp == "7":
                self.save_tracker()
            elif resp == "8":
                self.quit()
            else:
                print("Invalid response. Please try again.")
    
    def run(self):
        # upon running, if there isn't a data folder created, make one
        # load the tracker dict so later we can check to see if there's a conflict if
        # someone tries to use the same username
        
        if not os.path.isdir("data"):
            os.makedirs("data")
            
        # then check if there's a pickle file for the tracker names
        # if so, load the most recent one and assign to tracker_names
        
        pickles = ["data/" + f for f in os.listdir("data") if f[0:13] == "tracker_names"]
        
        if len(pickles) > 0:

            newest = max(pickles , key = os.path.getctime)
            
            with open(newest, "rb") as f:
                self.tracker_names = pickle.load(f)
        
        self.display_main()
        
        
    
    def create_tracker(self):
        """
        Creates a new tracker object if username/tracker doesn't already exist
        """
        print("Let's make a new Tracker!")
        
        while True:
            
            # TO DO: should build in a check for username conflicts
            username = input("Enter a username to associate with the Tracker: ")
            
            if username in self.tracker_names:
                print("""
                Username already exists in system. 
                
                1) Return to main menu to load tracker associated with that username
                2) Select another username               
                """)
                
                resp = input("Enter a response: ")
                
                if resp == "1":
                    # run display menu
                    pass
                elif resp == "2":
                    continue
            
            else:
                break
            
        # initialize Tracker
        tracker = Tracker(username)

        # add tracker to menu's tracker names
        self.tracker_names.add(username)
        # set this new tracker as the current tracker
        self.cur_tracker = tracker
        self.cur_username = username                  
                  
    
    def load_tracker(self):
        """
        Checks if inputted username is in tracker_names, which is loaded and assigned 
        based on whether a pickle file for the tracker name exists. If does exist, 
        then load the most recent tracker pickle file associated with that name inside
        the data directory
        """
        while True:
            
            username = input("Enter the username of the tracker you want to load: ")
            
            if username not in self.tracker_names:
                print("""
                Sorry, there is no saved tracker for that username. 
                
                1) Re-enter the username
                2) Create a new tracker
                """)
                
                resp = input("What would you like to do? ")
                
                if resp == '1':
                    continue
                elif resp == '2':
                    self.create_tracker()
                else:
                    print("Invalid response. Try again.")
            else:
                break
        
        # find the latest file of the corresponding username
        
        # set current username to username
        self.cur_username = username
        
        # make a list of all the files that correspond to username
        pickles = ["data/" + f for f in os.listdir("data") if f[0:len(username)] == username]
        
        # if there's at least one file, find the most recent one
        # and load it as the current tracker
        
        if len(pickles) > 0:

            newest = max(pickles , key = os.path.getctime)
            
            with open(newest, "rb") as f:
                self.cur_tracker = pickle.load(f)   
                print("Tracker successfully loaded!")
            
            # TO DO: launch display_functions
            
    def add_item_dates(self):
        
        """
        Asks user if they want to override date timestamp
        Checks if the date inputs are correct
        Returns formatted date object for the Tracker.add_item()
        """
        
        while True:
            
            print("""
            When an item is added to the wishlist, it is timestamped to today's date
            by default. You have the option to override this and input a custom date.
            For example, if you were tracking items elsewhere. Note: Item dates are used
            to calculate point accummulation so altering them will affect your points.     
                  """)
            resp = input("Do you want to override the item date? y/n ")
            
            if resp[0].lower() == "y":
                
                while True:
                    str_date = input("Enter a custom item date in the following format (MM/DD/YYYY): ")
                    
                    if str_date # include regex for matching
                        # if the format is correct, split the string and create the date object and return it
                    # if it doesn't match the format
                        # force them to input it again
                        # QUESTION: add a quit/exit function? or just break so it passes the control
                        
                
            elif resp[0].lower() == "n":
                
                return None
            
            else:
                print("Invalid response. Please try again.")
                continue
                
    def add_item_price(self):
        
        """
        Asks user to enter price of the item
        Performs error checking
        Returns the object
        """
        pass
    
    def add_item_cat(self):
        
        """
        Asks user to select a category for the item
        Performs error checking
        Returns the object
        """
        
        # can write a dict for the item categories here
        
        pass
    
    def add_item_value(self):
        
        """
        Asks user to enter a point value for the item
        Performs error checking (no negatives, nothing too high)
        Returns object
        """
        pass
          
    def add_item_run(self):
        """
        Adds an item to the wishlist utilizing Tracker function after
        running the functions to error check inputs to the Tracker.add_item() function
        """
        
        while True:
            
            print("To get started with adding an item, enter the item name.")
            
            item = input()
            
            if item in self.cur_tracker.wishlist.keys():
                # TO DO: add functionality to go to the update
                print("""
                This item already exists in the wishlist. 
                1) Enter a new item
                2) Update existing item 
                """)
                resp = input("What would you like to do? ")
                
                if resp == "1":
                    continue
                elif resp == "2":
                    #  QUESTION: possible to pass in what item they inputted previously?
                    # would make the transition more seamless
                    self.update_item()
                else:
                    continue
            else:
                break
                
            
                
            
            
        
        
        pass
    
    def update_item(self):
        pass
    
    def del_item(self):
        pass
    
    def redeem_item(self):
        pass
    
    def print_wishlist(self):
        pass
    
    def tracker_info(self):
        
        """ Displays information about the tracker """
        
        # QUESTION: Update with more metadata like last save?
        self.cur_tracker.tracker_info()
    
    def save_tracker(self):
        """
        Saves the tracker names into a pickle file inside the data directory
        Saves the current tracker into a pickle file inside the data directory
        """
        now = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        
        names_file = "data/tracker_names_" + now + ".pickle"
        tracker_file = "data/" + self.cur_username + "_" + now + ".pickle"
        
        with open(names_file, "wb") as f:
            pickle.dump(self.tracker_names, f)
            
        with open(tracker_file, "wb") as f:
            pickle.dump(self.cur_tracker, f)
            
        print("Impulse Spending Tracker successfully saved.")
        self.quit()
        
    
    def quit(self):
        print("The program will now exit. Thank you for using the Impulse Tracker!")
        exit(0)
    
    

In [13]:
m = Menu()
m.run()
m.load_tracker()
m.tracker_info()

Enter the username of the tracker you want to load: TK
Tracker successfully loaded!
Wishlist contains: {}
Start Date of Tracker: 2021-05-04
Username is: TK


In [5]:
m = Menu()
m.run()
m.create_tracker()
m.save_tracker()

Let's make a new Tracker!
Enter a username to associate with the Tracker: TK

                Username already exists in system. 
                
                1) Return to main menu to load tracker associated with that username
                2) Select another username               
                
Enter a response: Elaine
Enter a username to associate with the Tracker: Eileen
Impulse Spending Tracker successfully saved. The program will now exit.
Thank you for using the Impulse Tracker!


SystemExit: 0

C:\Users\tkayt\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
m.cur_username

'Eileen'

In [7]:
if len(os.listdir("data")) > 0:
    pickles = ["data/" + f for f in os.listdir("data") if f[0:13] == "tracker_names"]
    newest = max(pickles, key = os.path.getctime)
    
with open(newest, "rb") as f:
    
    test = pickle.load(f)
    
print(test)
print(newest)
                

{'Eileen', 'TK', 'Elaine'}
data/tracker_names_20210504_221550.pickle


In [28]:
len(["data/" + f for f in os.listdir("data") if f[0:13] == "tracker_names"])


0

In [42]:
f = open("data/tracker_names_30042021_175113.txt", "a")
f.write("Now the file has more content!")
f.close()

In [31]:
t1 = Tracker("TK")

t1.tracker_info()

Wishlist contains: {}
Start Date of Tracker: 2021-04-28
Username is: TK


In [32]:
t1.add_item("Bose headphones", 299.00, "Electronics", 800, "yes", dt.date(2021, 1, 14))
t1.add_item("Gldn Celine earrings", 29.99, "Jewelry", 400, "yes", dt.date(2021, 1, 14))
t1.add_item("Kris Nations Huggie Hoop earrings", 50.00, "Jewelry", 400, "yes", dt.date(2021, 1, 14))
t1.add_item("Gentlewoman's Agreement Ring", 148.00, "Jewelry", 1000, "yes", dt.date(2021, 1, 14))
t1.add_item("Bliss moisturizer", 19.99, "Beauty/Skincare", 200, "yes", dt.date(2021, 4, 9))
t1.add_item("Watermelon face milk", 14.00, "Beauty/Skincare", 200, "yes", dt.date(2021, 4, 9))
t1.add_item("Leggings", 19.99, "Clothes", 200, "yes", dt.date(2021, 3, 25))
t1.add_item("Slippers", 15.99, "Clothes", 200, "yes", dt.date(2021, 4, 2))
t1.add_item("Salt Acid Fat Heat Cookbook", 18.84, "Books", 200, "yes", dt.date(2021, 3, 15))
t1.add_item("Colsie Bralette", 12.99, "Clothes", 200, "yes", dt.date(2021, 3, 28))
t1.add_item("OPI Nail Polish", 9.99, "Beauty/Skincare", 100, "yes", dt.date(2021, 3, 28))
t1.add_item("Pizza Cupcakes", 38.98, "Food", 600, "yes", dt.date(2021, 3, 25))
t1.add_item("Mushroom Jerky", 35.99, "Food", 600, "yes", dt.date(2021, 3, 25))
t1.add_item("Old Navy Embroidered Sweatshirt", 34.99, "Clothes", 500, "yes", dt.date(2021, 4, 11))
t1.add_item("Winky Lux Lip Stain", 15.99, "Beauty/Skincare", 300, "yes", dt.date(2021, 4, 11))
t1.add_item("Colourpop Lip Stain", 8.00, "Beauty/Skincare", 200, "yes", dt.date(2021, 4, 15))
t1.print_wishlist()

Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist
Item added to wishlist


,price,category,value,date,redeemed_dt,redeemed,del_ind
Bose headphones,299,Electronics,800,2021-01-14,1900-01-01,no,no
Gldn Celine earrings,29.99,Jewelry,400,2021-01-14,1900-01-01,no,no
Kris Nations Huggie Hoop earrings,50,Jewelry,400,2021-01-14,1900-01-01,no,no
Gentlewoman's Agreement Ring,148,Jewelry,1000,2021-01-14,1900-01-01,no,no
Bliss moisturizer,19.99,Beauty/Skincare,200,2021-04-09,1900-01-01,no,no
Watermelon face milk,14,Beauty/Skincare,200,2021-04-09,1900-01-01,no,no
Leggings,19.99,Clothes,200,2021-03-25,1900-01-01,no,no
Slippers,15.99,Clothes,200,2021-04-02,1900-01-01,no,no
Salt Acid Fat Heat Cookbook,18.84,Books,200,2021-03-15,1900-01-01,no,no
Colsie Bralette,12.99,Clothes,200,2021-03-28,1900-01-01,no,no


In [33]:
p, c = t1.calc()
t1.redeem_item("Watermelon face milk")
t1.redeem_item("Colourpop Lip Stain")
print(t1.points)
print(t1.print_wishlist())

Item marked as redeemed
Item marked as redeemed
335
                                   price         category value        date  \
Bose headphones                      299      Electronics   800  2021-01-14   
Gldn Celine earrings               29.99          Jewelry   400  2021-01-14   
Kris Nations Huggie Hoop earrings     50          Jewelry   400  2021-01-14   
Gentlewoman's Agreement Ring         148          Jewelry  1000  2021-01-14   
Bliss moisturizer                  19.99  Beauty/Skincare   200  2021-04-09   
Leggings                           19.99          Clothes   200  2021-03-25   
Slippers                           15.99          Clothes   200  2021-04-02   
Salt Acid Fat Heat Cookbook        18.84            Books   200  2021-03-15   
Colsie Bralette                    12.99          Clothes   200  2021-03-28   
OPI Nail Polish                     9.99  Beauty/Skincare   100  2021-03-28   
Pizza Cupcakes                     38.98             Food   600  2021-03-25   


In [59]:
print(t1.calc())

(319, 750.7300000000001)


In [ ]:
# starting to rewrite with item class

class Item():
    
    """
    Item object, which has characteristics of the wishlist item such as name, price, point value, etc.
    """
    
    def __init__(self, name, price, value, link = " ", override_dates = "no", date = dt.date(1900, 1, 1)):
        """
        Create object with base characteristics. Allows for overriding date logging, which defaults to
        current date in case we need to do back track entries.
        """
        
        self.name = name
        self.price = price
        self.value = value
        self.link = link
        self.redeemed_date = " "
        self.redeemed = "no"
        
        if override_dates == "no":
            
            self.date = dt.date.today()
            
        elif override_dates == "yes":
            
            self.date = date
            
    def item_update(self, )

